High Value Customer Identification (Insiders)

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import requests

In [48]:
!pip list --format=freeze > requirements.txt

# Multiple products

In [2]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text, "html.parser")

total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )
total_item

'91'

In [3]:
page_number = np.round( int( total_item ) / 36 )
page_number

3.0

In [4]:
urlfull = url + '?page-size=' + str( int( page_number*36 ) )
urlfull

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=108'

In [53]:
page = requests.get( urlfull , headers=headers )


soup = BeautifulSoup( page.text, "html.parser")

#total_item = p.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )

total_item = [p.get("data-articlecode") for p in soup.find_all("article", class_='hm-product-item')]

len( total_item )

90

In [54]:
products_ids = []

for i in total_item:

    url = f'https://www2.hm.com/en_us/productpage.{i}.html'

    page = requests.get( url, headers=headers )

    soup = BeautifulSoup( page.text , 'html.parser' )

    aux = soup.find_all('a', class_='filter-option miniature active') + soup.find_all('a', class_='filter-option miniature')
    aux = [p.get('data-articlecode') for p in aux]

    products_ids.append(aux)

len(products_ids)

90

In [55]:
products_ids2 = []

for i in products_ids:
    products_ids2 += i

products_ids2 = list(set(products_ids2))

products_ids2

['1071707010',
 '0993887008',
 '0938875010',
 '1018704002',
 '1027852007',
 '1013317010',
 '1063141001',
 '0690449046',
 '0811993025',
 '1114023003',
 '1008110006',
 '1071707001',
 '0690449002',
 '0938875014',
 '0875105029',
 '1099503002',
 '0993887007',
 '0985159002',
 '0811993048',
 '1024256001',
 '1024711001',
 '0811993006',
 '1096385001',
 '0971061004',
 '0811993039',
 '0875105003',
 '0993887004',
 '0811993040',
 '0690449011',
 '1008549003',
 '0690449021',
 '0927964005',
 '1024256004',
 '0690449001',
 '0690449035',
 '1114023001',
 '1018704003',
 '0985197002',
 '1024711004',
 '1027852009',
 '1013317004',
 '1024711002',
 '0811993007',
 '1008110002',
 '1013317002',
 '0938875013',
 '1107750002',
 '1071707008',
 '0971061001',
 '0690449064',
 '1004199007',
 '1025726002',
 '1074475003',
 '1099503001',
 '1008549002',
 '0985159007',
 '1004199002',
 '0690449009',
 '1024256007',
 '0985159006',
 '1071707002',
 '0811993028',
 '0927964013',
 '1100162002',
 '0690449028',
 '1025726003',
 '09388750

In [56]:
len( products_ids2 )

179

In [76]:
cols = ['product_id', 'product_name', 'composition', 'color', 'size', 'fit', 'additional material information']
df_pattern = pd.DataFrame( columns = cols)


for i in products_ids2:

    url = f'https://www2.hm.com/en_us/productpage.{i}.html'
    page = requests.get( url, headers=headers )
    soup = BeautifulSoup( page.text , 'html.parser' )

    product_list = soup.find_all('div', class_='details-attributes-list-item')

    # remove empty strings
    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_list]

    dici = {}
    for l2 in product_composition:
        dici[l2[0].lower()] = l2[1]




    name_price = soup.find_all('section', class_='product-name-price')[0]
    product_name = name_price.find('h1').text
    product_price = name_price.find('span', class_='price-value').text

    dici['product_name'] = product_name
    dici['price'] = product_price

    aux = soup.find_all('a', class_='filter-option miniature active')[0]
    dici['color'] = aux.get('data-color')


    df_aux = pd.DataFrame.from_dict(dici, orient='index').T
    df_aux = df_aux.rename(columns={'art. no.':"product_id"})

    df_pattern = pd.concat( [df_pattern,df_aux] )

df_pattern.head()

,product_id,product_name,composition,color,size,fit,additional material information,messages.garmentlength,messages.waistrise,care instructions,material,description,imported,concept,price,messages.clothingstyle,collection
0,1071707010,Relaxed Jeans,Shell: Cotton 100%,Denim gray,Inner leg: Length: 81.0 cm (Size 33/32),Relaxed fit,NaN,Long,Regular waist,Only non-chlorine bleach when needed,keyFibreType,Denim gray,Yes,DENIM,\r\n $29.99,NaN,NaN
0,0993887008,Hybrid Regular Jeans,"Cotton 78%, Polyester 21%, Spandex 1%",Dark gray,NaN,Regular fit,NaN,Long,Regular waist,Machine wash cool,Denim,Dark gray,Yes,DENIM,\r\n $39.99,NaN,NaN
0,0938875010,Slim Tapered Jeans,"Cotton 99%, Spandex 1%",Light blue,NaN,Slim fit,NaN,Long,Regular waist,Only non-chlorine bleach when needed,keyFibreType,Light blue,Yes,DENIM,\r\n $22.99,Tapered,NaN
0,1018704002,Relaxed Pull-on Jeans,Shell: Cotton 100%,Denim blue,Inner leg: Length: 81.3 cm (Size L),Relaxed fit,Recycled cotton 20%,Long,Regular waist,Tumble dry medium,keyFibreType,Denim blue,Yes,DENIM,\r\n $24.99,Pull-on Pants,NaN
0,1027852007,Relaxed Denim Joggers,Shell: Cotton 100%,Denim blue,Inner leg: Length: 77.7 cm (Size L),Relaxed fit,NaN,Long,Regular waist,Tumble dry medium,keyFibreType,Denim blue,Yes,DENIM,\r\n $29.99,Joggers,NaN


In [77]:
df_pattern2 = df_pattern.copy()
df_pattern2['style_id'] = df_pattern2['product_id'].str[:-3]
df_pattern2['color_id'] = df_pattern2['product_id'].str[-3:]
df_pattern2 = df_pattern2.sort_values('product_id')
df_pattern2['scrapy_datetime']=datetime.today()
df_pattern2.head()

,product_id,product_name,composition,color,size,fit,additional material information,messages.garmentlength,messages.waistrise,care instructions,material,description,imported,concept,price,messages.clothingstyle,collection,style_id,color_id,scrapy_datetime
0,0690449001,Skinny Jeans,"Cotton 99%, Spandex 1%",Light denim blue/trashed,NaN,Skinny fit,NaN,Long,Regular waist,Machine wash cool,keyFibreType,Light denim blue,Yes,DENIM,\r\n $13.99,Trashed,NaN,0690449,001,2022-12-04 07:39:13.070636
0,0690449002,Skinny Jeans,"Cotton 98%, Spandex 2%",Denim blue,NaN,Skinny fit,NaN,Long,Regular waist,Machine wash cool,keyFibreType,Denim blue,Yes,DENIM,\r\n $14.99,Trashed,NaN,0690449,002,2022-12-04 07:39:13.070636
0,0690449006,Skinny Jeans,"Cotton 98%, Spandex 2%",Black/washed,NaN,Skinny fit,NaN,Long,Regular waist,Machine wash cool,keyFibreType,Black,Yes,DENIM,\r\n $7.99,Trashed,NaN,0690449,006,2022-12-04 07:39:13.070636
0,0690449007,Skinny Jeans,"Cotton 98%, Spandex 2%",Light denim blue,NaN,Skinny fit,NaN,Long,Regular waist,Tumble dry medium,keyFibreType,Light blue,Yes,DENIM,\r\n $14.99,Trashed,NaN,0690449,007,2022-12-04 07:39:13.070636
0,0690449009,Skinny Jeans,"Cotton 99%, Spandex 1%",Black washed out,NaN,Skinny fit,NaN,Long,Regular waist,Machine wash cool,keyFibreType,Black,Yes,DENIM,\r\n $17.99,Trashed,NaN,0690449,009,2022-12-04 07:39:13.070636


In [78]:

df_details = pd.DataFrame()

#unique columns for all products
aux = []

cols = [ 'product-name', 'composition', 'color', 'size', 'fit', 'article-code']
df_pattern = pd.DataFrame( columns = cols)

for i in products_ids2[:2]:

    url = f'https://www2.hm.com/en_us/productpage.{i}.html'

    page = requests.get( url, headers=headers )

    soup = BeautifulSoup( page.text , 'html.parser' )

    product_list = soup.find_all('div', class_='details-attributes-list-item')


    # remove empty strings
    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_list]


    # list to dataframe
    data_composition = pd.DataFrame(product_composition).T


    # data aux
    data_composition_aux = data_composition.iloc[0]


    # rename df columns
    data_composition.columns = data_composition_aux

    aux = aux + data_composition.columns.tolist()

    aux = [i for i in aux if "messages." not in i]

    # forward fillna
    data_composition = data_composition.iloc[1:].fillna(method='ffill')

    data_composition['Composition'] = data_composition['Composition'].str.replace( "Shell: |Pocket lining: |Lining","", regex=True )

    data_composition = pd.concat( [df_pattern, data_composition], axis=0)


    # drop duplicates
    data_composition = data_composition.drop_duplicates(subset=['Fit', 'Composition', 'Art. No.'], keep='first', inplace=False, ignore_index=False)


    # only selected columns
    data_composition = data_composition[cols].copy()

    data_composition.columns = data_composition.columns.str.lower()

    data_composition['style_id'] = data_composition['art. no.'].str[:-3]

    data_composition['color_id'] = data_composition['art. no.'].str[-3:]



    # Produto Final
    df_details = pd.concat( [df_details, data_composition] , axis=0)

KeyError: 'art. no.'

In [68]:
df_pattern2.to_csv("df_products.csv", index=False)